In [ ]:
!pip install keras-tuner

In [2]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("diabetes.csv")

In [5]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [6]:
data.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [7]:
X = data.drop('Outcome', axis=1)

In [8]:
y = data['Outcome']

In [9]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=48)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)


In [12]:
X_test = scaler.transform(X_test)

In [13]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad, SGD

In [14]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(8,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values = ['RMSprop', 'Adam', 'Adagrad', 'SGD'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [15]:
# Tuner object for KT class
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [16]:
tuner.search(X_train, y_train, epochs=10,validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.5974025726318359

Best val_accuracy So Far: 0.701298713684082
Total elapsed time: 00h 00m 10s


In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'SGD'}

In [18]:
best_model = tuner.get_best_models(num_models=1)[0]

In [19]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Will fit the best model now from the Initial_epoch as 11 as we already ran 10 previosly.
best_model.fit(X_train, y_train, epochs=100, initial_epoch=11,validation_data=(X_test, y_test))

Epoch 12/100
20/20 [==============================] - 1s 21ms/step - loss: 0.5870 - accuracy: 0.6596 - val_loss: 0.5673 - val_accuracy: 0.6948
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5797 - accuracy: 0.6678 - val_loss: 0.5602 - val_accuracy: 0.7013
Epoch 14/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5731 - accuracy: 0.6694 - val_loss: 0.5540 - val_accuracy: 0.7078
Epoch 15/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5674 - accuracy: 0.6792 - val_loss: 0.5476 - val_accuracy: 0.7078
Epoch 16/100
20/20 [==============================] - 0s 10ms/step - loss: 0.5617 - accuracy: 0.6808 - val_loss: 0.5419 - val_accuracy: 0.7078
Epoch 17/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5565 - accuracy: 0.6857 - val_loss: 0.5365 - val_accuracy: 0.7143
Epoch 18/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5517 - accuracy: 0.6857 - val_loss: 0.5318 - val_accuracy: 0.7143
Epoc

In [28]:
# Selecting No. of Nodes in a layer
def build_model(hp):
  model = Sequential()

  units = hp.Int('units', min_value=4,max_value=96,step=4)

  model.add(Dense(units=units, activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='Adam',metrics=['accuracy'], loss='binary_crossentropy')

  return model

In [29]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        project_name = 'keras-tuner'
                        )

In [30]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 10s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'units': 56}

In [32]:
best_model = tuner.get_best_models(num_models=1)[0]

In [33]:
best_model.fit(X_train, y_train, epochs=100, initial_epoch=11, validation_data=(X_test, y_test))

Epoch 12/100
20/20 [==============================] - 1s 11ms/step - loss: 0.4712 - accuracy: 0.7720 - val_loss: 0.4647 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4664 - accuracy: 0.7785 - val_loss: 0.4637 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4627 - accuracy: 0.7801 - val_loss: 0.4631 - val_accuracy: 0.7727
Epoch 15/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4585 - accuracy: 0.7850 - val_loss: 0.4618 - val_accuracy: 0.7857
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4562 - accuracy: 0.7818 - val_loss: 0.4624 - val_accuracy: 0.7857
Epoch 17/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4539 - accuracy: 0.7866 - val_loss: 0.4630 - val_accuracy: 0.7857
Epoch 18/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4511 - accuracy: 0.7899 - val_loss: 0.4618 - val_accuracy: 0.7662
Epoch

In [58]:
#For selecting no of Layers, no. of Nodes, Optimizes and Activation function
def build_model(hp):

   model = Sequential()

   count =0

   for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

     if count == 0:
       model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                       activation = hp.Choice('activation' + str(i), values=['sigmoid','relu','tanh','elu']),
                       input_dim=8
                       )
       )
     else:

       model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                       activation = hp.Choice('activation' + str(i), values=['sigmoid','relu','tanh','elu']),
                       )
       )
       count+=1

   model.add(Dense(1, activation='sigmoid'))

   model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam','adagrad','adadelta','sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
   return model


In [59]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=8,
                        directory = 'Tuner',
                        project_name='Final_Tuning')

In [60]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 8 Complete [00h 00m 04s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 09s


In [61]:
# using 10 epochs
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 104,
 'activation0': 'sigmoid',
 'optimizer': 'rmsprop',
 'units1': 104,
 'activation1': 'tanh',
 'units2': 56,
 'activation2': 'sigmoid',
 'units3': 48,
 'activation3': 'sigmoid',
 'units4': 16,
 'activation4': 'relu',
 'units5': 40,
 'activation5': 'tanh',
 'units6': 80,
 'activation6': 'tanh',
 'units7': 128,
 'activation7': 'elu',
 'units8': 88,
 'activation8': 'elu',
 'units9': 40,
 'activation9': 'relu'}

In [62]:
final_model = tuner.get_best_models(num_models=1)[0]

In [63]:
final_model.fit(X_train, y_train, epochs=100, initial_epoch=11, validation_data=(X_test, y_test))

Epoch 12/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4787 - accuracy: 0.7508 - val_loss: 0.4879 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4851 - accuracy: 0.7671 - val_loss: 0.4903 - val_accuracy: 0.7727
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4807 - accuracy: 0.7638 - val_loss: 0.4703 - val_accuracy: 0.7857
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4719 - accuracy: 0.7752 - val_loss: 0.4893 - val_accuracy: 0.7727
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4825 - accuracy: 0.7801 - val_loss: 0.5260 - val_accuracy: 0.7403
Epoch 17/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4754 - accuracy: 0.7704 - val_loss: 0.4966 - val_accuracy: 0.7468
Epoch 18/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4870 - accuracy: 0.7606 - val_loss: 0.5207 - val_accuracy: 0.7792
Epoch